In [6]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def selecionar_melhor_k(ks, X_treino, X_val, y_treino, y_val):
    acuracias_val = []
    for k in ks:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_treino, y_treino)
        pred = knn.predict(X_val)
        acuracias_val.append(accuracy_score(y_val, pred))

    melhor_val = max(acuracias_val)
    melhor_k = ks[np.argmax(acuracias_val)]

    knn = KNeighborsClassifier(n_neighbors=melhor_k)
    knn.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])
    
    return knn, melhor_k, melhor_val

In [3]:
ds = load_iris(as_frame=True)
df = ds.frame
X = ds.data.values
y = ds.target.values

In [19]:
folds = 10
skf = StratifiedKFold(n_splits=folds)

acuracias = []

for idx_treino, idx_teste in skf.split(X,y):
    X_treino = X[idx_treino]
    y_treino = y[idx_treino]

    X_teste = X[idx_teste]
    y_teste = y[idx_teste]
    
    X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.2, stratify=y_treino, random_state = 1)
    
    ss=StandardScaler()
    ss.fit(X_treino)
    X_treino = ss.transform(X_treino)
    X_val = ss.transform(X_val)
    X_teste = ss.transform(X_teste)
    
    knn,_,_ = selecionar_melhor_k(range(1,30,2), X_treino,X_val,y_treino,y_val)
    
    acuracias.append(accuracy_score(y_teste,knn.predict(X_teste)))


In [21]:
"%.2f +- %.2f (min: %.2f, max: %.2f)" % (np.mean(acuracias), np.std(acuracias), min(acuracias), max(acuracias))

'0.95 +- 0.07 (min: 0.80, max: 1.00)'

In [22]:
acuracias

[1.0,
 0.9333333333333333,
 1.0,
 0.9333333333333333,
 0.8666666666666667,
 1.0,
 0.8,
 1.0,
 1.0,
 1.0]